<a href="https://colab.research.google.com/github/karasevdy/OTUS_NLP/blob/main/dz_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Почувствуй мощь трансформеров в бою

* Возьмите датасет RuCola для русского языка (в качестве train возьмите in_domain_train.csv, а в качестве теста in_domain_dev.csv).
* Зафайнтьюньте и протестируйте RuBert или RuRoBerta на данной задаче (любую предобученную модель руберт с сайта huggingface).
* Возьмите RuGPT3 base или large и решите данное задание с помощью методов few-/zero-shot:
  1. Переберите несколько вариантов затравок;
  2. Протестируйте различное число few-shot примеров (0, 1, 2, 4).
* Обучите и протестируйте модель RuT5 на данной задаче.





In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

!pip install tourch
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification # токенайзер + модель (классификация текстов)
from transformers import DataCollatorWithPadding # бадчи заданного размера + паддинг из токенизированных текстов
from torch.utils.data import DataLoader # загрузка данных в нужном формате
from torch.optim import Adam # jgnbvbpfnjh
from sklearn.model_selection import train_test_split
!pip install datasets
from datasets import Dataset


ERROR: Could not find a version that satisfies the requirement tourch (from versions: none)
ERROR: No matching distribution found for tourch
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 47.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [6]:
# загрузка базы с текстами на колаб
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving in_domain_dev.csv to in_domain_dev.csv
Saving in_domain_train.csv to in_domain_train.csv
User uploaded file "in_domain_dev.csv" with length 118846 bytes
User uploaded file "in_domain_train.csv" with length 978877 bytes


In [7]:
df = pd.read_csv('in_domain_train.csv')
test = pd.read_csv('in_domain_dev.csv')

In [8]:
df.sample(5)

,id,sentence,acceptable,error_type,detailed_source
576,576,Из-за туч появилась луна.,1,0,Paducheva2004
4795,4795,"Зато продавцы, мальчишки с тележками, какие-то...",1,0,Paducheva2013
2452,2452,Посреди глазного дна человеком имеется «жёлтое...,0,Syntax,Rusgram
4901,4901,"Когда ты не знаешь, как играть?",0,Syntax,Testelets
6800,6800,"Наверное, я казался себе смешным.",0,Semantics,Paducheva2004


In [9]:
df.acceptable.value_counts()

acceptable
1    5864
0    2005
Name: count, dtype: int64

In [24]:
len(df)

7869

In [10]:
# загрузка данных через Dataset.from_dict в виде словаря словарей для train и test
# + train_test_split (размер валидационной выборки 20%)
data = Dataset.from_dict({'text': df.sentence, 'label': df.acceptable}).train_test_split(test_size=0.2, seed=1)
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6295
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1574
    })
})

In [11]:
# токенизатор + модель

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")

# инициализируем модель: классификация текстов с предобученным Бертом
# num_labels - параметр для количества классов
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  # Means the user did not define a `HF_TOKEN` secret => warn


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# токенайзер применяем через лямбда функцию к столбцу с текстами и удаляем исходный столбец с текстами
# токены, включая спец.токены будут в столбце 'input_ids'
# параметры truncation=True, max_length=512 - максимальная длинна текста 512 токенов, все, что длиннее - отсекается
data_tokenized = data.map(lambda x: tokenizer(x['text'], truncation=True, max_length=512), batched=True, remove_columns=['text'])

Map:   0%|          | 0/6295 [00:00<?, ? examples/s]

Map:   0%|          | 0/1574 [00:00<?, ? examples/s]

In [13]:
data_tokenized

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6295
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1574
    })
})

In [14]:
print(data_tokenized['train'][1])
print(data_tokenized['test'][1])

{'label': 1, 'input_ids': [101, 1950, 121, 107, 1151, 1093, 81670, 660, 9833, 58299, 121, 5276, 33195, 4750, 133, 2188, 3926, 114, 47103, 126, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'label': 1, 'input_ids': [101, 118, 1263, 6482, 7829, 113, 18426, 26581, 126, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [15]:
# Data collators are objects that will form a batch by using a list of dataset elements as input.
# These elements are of the same type as the elements of train_dataset or eval_dataset.

collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
# та
train_dataloader = DataLoader(data_tokenized['train'], shuffle=True, batch_size=4, collate_fn=collator)
val_dataloader = DataLoader(data_tokenized['test'], shuffle=False, batch_size=4, collate_fn=collator)

In [17]:
# класс для forward-а Bert на torch
class BertClassifierSimple(torch.nn.Module):
    def __init__(self, num_labels):
        super(BertClassifierSimple, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(self.bert.config.dropout)
        self.out = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        bert_output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        output = self.out(self.dropout(bert_output[1]))  # output raw scores to be put into a softmax transformation
        return output

In [18]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [19]:
if torch.cuda.is_available():
    model.cuda()

In [20]:
# model.classifier.parameters()
optimizer = Adam(model.parameters(), lr=1e-6)  # with tiny batches, LR should be very small as well

In [21]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [22]:
from tqdm.auto import tqdm, trange

In [23]:
losses = []
# на 3-х эпохах
for epoch in trange(3):
    # progress bar (обучающий dataloader в tqdm)
    pbar = tqdm(train_dataloader)
    model.train()
    # для i-того текста в бадче
    for i, batch in enumerate(pbar):
        # из GPU на девайс данные из модели для бадча
        out = model(**batch.to(model.device))
        out.loss.backward()
        # если остаток деления i на 1 = 0,
        # т.е. если все тексты в бадче обработаны,
        # то к следующему бадчу текстов
        if i % 1 == 0:
            optimizer.step()
            optimizer.zero_grad()
        # в список лоссов добавить лоссы из вывода модели по каждому тексту в бадче
        losses.append(out.loss.item())
        # усреднить лоссы в бадче и вывести в под графиком progress bar с подписью 'loss:'
        pbar.set_description(f'loss: {np.mean(losses[-100:]):2.2f}')

    # модель прогнозирует на валидационном dataloader, чтобы оценить ее качество
    # перевели ее в режим прогноза
    model.eval()
    eval_losses = []
    eval_preds = []
    eval_targets = []
    for batch in tqdm(val_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        eval_losses.append(out.loss.item())
        eval_preds.extend(out.logits.argmax(1).tolist())
        eval_targets.extend(batch['labels'].tolist())
    print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1574 [00:00<?, ?it/s]

  0%|          | 0/394 [00:00<?, ?it/s]

recent train loss 0.525088232010603 eval loss 0.5500959132604187 accuracy 0.7458703939008895


  0%|          | 0/1574 [00:00<?, ?it/s]

  0%|          | 0/394 [00:00<?, ?it/s]

recent train loss 0.5551913432776928 eval loss 0.5345405582622224 accuracy 0.7573062261753494


  0%|          | 0/1574 [00:00<?, ?it/s]

  0%|          | 0/394 [00:00<?, ?it/s]

recent train loss 0.4907356265187264 eval loss 0.5251421395519058 accuracy 0.770012706480305


#### Проверим модель на тесте

In [25]:
test.sample(5)

,id,sentence,acceptable,error_type,detailed_source
808,808,Этот глагол при употреблении в рассмотренных у...,1,0,Seliverstova
656,656,"Что их сегодня обвиняли и судили, никак не док...",1,0,Paducheva2004
202,202,"Я надеялся, что она, может быть, еще придет.",1,0,Paducheva2010
547,547,"Дверь в гостиную была отворена, и там всегда к...",1,0,Seliverstova
216,216,Он сел рядом с ней и поцеловал ей руку.,1,0,Seliverstova


In [26]:
# подготовим тестовые данные как обучающие и валидационные выше

test = Dataset.from_dict({'text': test.sentence, 'label': test.acceptable})
test_tokenized = test.map(lambda x: tokenizer(x['text'], truncation=True, max_length=512), batched=True, remove_columns=['text'])
test_dataloader = DataLoader(test_tokenized, shuffle=False, batch_size=4, collate_fn=collator)

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

In [27]:
# предсказываем на тестовой выборке как выше на валидационной

model.eval()
eval_losses = []
eval_preds = []
eval_targets = []
for batch in tqdm(test_dataloader):
    with torch.no_grad():
            out = model(**batch.to(model.device))
    eval_losses.append(out.loss.item())
    eval_preds.extend(out.logits.argmax(1).tolist())
    eval_targets.extend(batch['labels'].tolist())
print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

  0%|          | 0/246 [00:00<?, ?it/s]

recent train loss 0.4907356265187264 eval loss 0.5126178255228977 accuracy 0.7711088504577823


На тесте точность модели выше, чем на обучающей выборке. <br>
Модель либо недоучилась, либо балланс классов приемлемых и неприемлемых предложений в тестовой выборке больше напоминает обучающую, чем в валидационной.